In [11]:
import requests
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas
from selenium.webdriver.chrome.options import Options
import beepy
import time
from datetime import datetime,timedelta

In [18]:
def find_appointment_at_location(city = "Camden",hours_to_run_the_script = 8, max_possible_wait_days = 30):
    
    max_time = time.time() + hours_to_run_the_script*60*60
    
    while time.time() < max_time:
        
        options = Options()
        options.headless = True
        driver = webdriver.Chrome(options = options)
        driver = webdriver.Chrome(options = options)
        driver.get("https://telegov.njportal.com/njmvc/AppointmentWizard/17")
        a = driver.execute_script("return locationData")
        b = driver.execute_script("return timeData")
        c = driver.execute_script("return locationModel")
        
        df_a = pandas.DataFrame.from_records(a)
        df_b = pandas.DataFrame.from_records(b)
        
        df_b[["FirstOpenSlot","Next Available"]] = df_b['FirstOpenSlot'].str.split('<br/>', expand = True)
        df_b["FirstOpenSlot"] = df_b['FirstOpenSlot'].str.split(' ', expand = True)
        df_b[["Dummy","Next Available"]] = df_b['Next Available'].str.split(': ', expand = True)
        df_b.drop(["Dummy"],axis = 1, inplace = True)
        
        final_df = pandas.merge(left = df_b, right = df_a, how = 'left', left_on = 'LocationId', right_on = 'Id')
        final_df.index = final_df["City"]
        final_df = final_df[["FirstOpenSlot","Next Available"]]
        
        if final_df.loc[city]["Next Available"]:
            
            latest_available_timeslot = datetime.strptime(final_df.loc[city]["Next Available"], '%m/%d/%Y %I:%M %p')
            max_possible_acceptable_date = datetime.today() + timedelta(days = max_possible_wait_days)
            min_possible_acceptable_date = datetime.today() + timedelta(days = max_possible_wait_days - 15)
            
        
            if (latest_available_timeslot < max_possible_acceptable_date) and (latest_available_timeslot > min_possible_acceptable_date): 
                print("Appointment available")
                beepy.beep(sound = 'coin')
                break
            else:
                print("Nothing Yet",latest_available_timeslot,final_df.loc[city]["FirstOpenSlot"])
                pass    
        
        else:
            print("This center has no appointments")
            
        time.sleep(5)
        
    return None

In [ ]:
find_appointment_at_location("Bayonne")